In [4]:
#this script obtains demographic data from the 2010 census summary file 1. 
#The end goal is creating bson documents to upload into mongo db with
#population counts by age,race,and sex for each area. For now I'll stick with
#the county level to keep things simple. 

#Note: need mdbtools installed on your computer.

Sample of JSON document I am hoping to create with this script...

[
    {
    "_id" : "asdfasdfa102838",
    "LOGRECNO" : "000023445",
    "FIPS" : "55025",
    "name" : "Dane County",
    "age" : "15-17",
    "sex" : "Male",
    "race" : "Asian",
    "count" : 26541,
    }
];

    

In [5]:
#load packages, set working directory, and load data:
library(dplyr)
library(RCurl)
library(readr)
library(Hmisc)#using mdb.get from here.
library(magrittr)#use %<>% from here
library(stringr)


root <- "~/Desktop"#change if you'd like. 
remotePath <- "http://www2.census.gov/census_2010/04-Summary_File_1/"
setwd(root)

# data from http://www2.census.gov/census_2010/04-Summary_File_1/Wisconsin/wi2010.sf1.zip

#select state and create new folder for that state's data. 
state <- "Wisconsin"
abb <- "wi"#to do: automate this later to abbreviate each state instead of hand labelling. 
if(!dir.exists("summary-files")) dir.create("summary-files")
setwd("summary-files")

if(!dir.exists(state)) dir.create(state)
setwd(state)

temp <- tempfile()
if(!file.exists("wi000012010.sf1")) {#download and unzip files if not already in directory
    download.file(paste0(remotePath, state,"/", paste0(abb,"2010.sf1.zip")),
                                                           destfile = temp, quiet = TRUE, method = "curl")
    unzip(temp)
    }
#now all state demographic files should be in working directory.     
#pct12 contains all age,sex,and race totals in summary file 1.
    

------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: bitops
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: 'Hmisc'

The following objects are masked from 'package:dplyr':

    combine, src, summarize

The following objects are masked from 'package:base':

    format.pval, round.POSIXt, trunc.POSIXt, units



In [6]:
#Obtain Feature Labels from MS Access files:
if(!file.exists("SF1_Access2003.mdb")) download.file("http://www2.census.gov/census_2010/04-Summary_File_1/SF1_Access2003.mdb",
              destfile = "SF1_Access2003.mdb",quiet=TRUE,method="curl")
if(!file.exists("DPSF2010_Access2003.mdb")) download.file("http://www2.census.gov/census_2010/03-Demographic_Profile/DPSF2010_Access2003.mdb",
              destfile = "DPSF2010_Access2003.mdb",quiet=TRUE,method="curl")
    
mdb.get("SF1_Access2003.mdb",tables="DATA_FIELD_DESCRIPTORS") -> descriptions #names corresponding to demographic headers.
mdb.get("SF1_Access2003.mdb",tables="SF1_00001") -> demHeader
mdb.get("SF1_Access2003.mdb",tables="SF1_00007mod") -> demNames_07 #These contain all of the demographic headers down to 
mdb.get("SF1_Access2003.mdb",tables="SF1_00008mod") -> demNames_08 ##the block level
#mdb.get("SF1_Access2003.mdb",tables="GEO_HEADER_SF1") -> geoHeaders #The geographic information for each row of data.    
mdb.get("DPSF2010_Access2003.mdb", tables="Header") -> geoHeaders ####might not even need this.
dem07 <- cbind(demHeader[1:4], demNames_07)
dem08 <- cbind(demHeader[1:4], demNames_08)
summary(descriptions)
head(dem07)
head(dem08)
head(geoHeaders)

    SORT.ID        SEGMENT       TABLE.NUMBER                     FIELD.NAME  
 Min.   :   1   Min.   : 1.00   PCT12  : 211   Total:                  : 220  
 1st Qu.:2404   1st Qu.:11.00   PCT12A : 211             20 years      :  72  
 Median :4808   Median :23.00   PCT12B : 211             21 years      :  72  
 Mean   :4808   Mean   :23.25   PCT12C : 211             10 to 14 years:  60  
 3rd Qu.:7211   3rd Qu.:34.00   PCT12D : 211             25 to 29 years:  60  
 Max.   :9614   Max.   :47.00   PCT12E : 211             30 to 34 years:  60  
                                (Other):8348   (Other)                 :9070  
     FIELD.CODE      DECIMAL      
          : 702   Min.   :0.0000  
 H00010001:   1   1st Qu.:0.0000  
 H0020001 :   1   Median :0.0000  
 H0020002 :   1   Mean   :0.0237  
 H0020003 :   1   3rd Qu.:0.0000  
 H0020004 :   1   Max.   :2.0000  
 (Other)  :8907   NA's   :739     

Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

FILEID,STUSAB,CHARITER,CIFSN,LOGRECNO,P0500001,P0500002,P0500003,P0510001,P0510002,⋯,P012E040,P012E041,P012E042,P012E043,P012E044,P012E045,P012E046,P012E047,P012E048,P012E049


Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(f_parts$left, ellip_h, f_parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

FILEID,STUSAB,CHARITER,CIFSN,LOGRECNO,P012F001,P012F002,P012F003,P012F004,P012F005,⋯,P016G003,P016H001,P016H002,P016H003,P016I001,P016I002,P016I003,P017A001,P017A002,P017A003


DESC,NAME,LEN
File Identification,FILEID,6
State/US-Abbreviation (USPS),STUSAB,2
Summary Level,SUMLEV,3
Geographic Component,GEOCOMP,2
Characteristic Iteration,CHARITER,3
Characteristic Iteration File Sequence Number,CIFSN,2


In [8]:
#Merge headers to data files:

if(!file.exists(paste0(abb,"000072010.sf1"))) paste0(abb,"000072010.sf1") %>% read_csv(col_names=FALSE,
                                          col_types = cols(.default = col_character())) -> sf_07
if(!file.exists(paste0(abb,"000082010.sf1"))) paste0(abb,"000082010.sf1") %>% read_csv(col_names=FALSE,
                                          col_types = cols(.default = col_character())) -> sf_08
if(!file.exists(paste0(abb,"geo2010.sf1"))) read_fwf(paste0(abb,"geo2010.sf1"), fwf_widths(geoHeaders$LEN,col_names=as.character(geoHeaders$NAME)),
         col_types = cols(.default = col_character()))-> geoData



In [9]:
geo <- geoData %>% filter(SUMLEV=="050")#county level data
head(geo)

FILEID,STUSAB,SUMLEV,GEOCOMP,CHARITER,CIFSN,LOGRECNO,REGION,DIVISION,STATE,⋯,SLDL2,SLDL3,SLDL4,AIANHHSC,CSASC,CNECTASC,MEMI,NMEMI,PUMA,RESERVED
SF1ST,WI,050,00,000,NA,0000020,2,3,55,,NA,NA,NA,NA,00,NA,9,NA,NA,NA
SF1ST,WI,050,00,000,NA,0002473,2,3,55,,NA,NA,NA,NA,00,NA,9,NA,NA,NA
SF1ST,WI,050,00,000,NA,0004915,2,3,55,,NA,NA,NA,NA,00,NA,9,NA,NA,NA
SF1ST,WI,050,00,000,NA,0008619,2,3,55,,NA,NA,NA,NA,00,NA,9,NA,NA,NA
SF1ST,WI,050,00,000,NA,0011746,2,3,55,,NA,NA,NA,NA,00,NA,1,NA,NA,NA
SF1ST,WI,050,00,000,NA,0018034,2,3,55,,NA,NA,NA,NA,00,NA,9,NA,NA,NA


In [10]:
geo1 <- geo %>% select(STUSAB, SUMLEV, GEOCOMP, LOGRECNO, STATE, COUNTY, COUNTYCC, NAME)
names(sf_07) <- names(dem07)
names(sf_08) <- names(dem08)
head(sf_07)

FILEID,STUSAB,CHARITER,CIFSN,LOGRECNO,P0500001,P0500002,P0500003,P0510001,P0510002,⋯,P012E040,P012E041,P012E042,P012E043,P012E044,P012E045,P012E046,P012E047,P012E048,P012E049
SF1ST,WI,000,07,0000001,5474377,61643,5412734,150214,136706,,56,47,13,21,6,12,6,4,3,4
SF1ST,WI,000,07,0000002,37100,382,36718,261,239,,0,2,0,0,0,0,0,0,0,0
SF1ST,WI,000,07,0000003,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0
SF1ST,WI,000,07,0000004,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0
SF1ST,WI,000,07,0000005,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0
SF1ST,WI,000,07,0000006,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0


In [45]:
library(data.table)
sfa <- data.table(sf_07, key="LOGRECNO")
sfb <- data.table(sf_08, key="LOGRECNO")
geo_table <- data.table(geo1, key = "LOGRECNO")
head(geo_table)

STUSAB,SUMLEV,GEOCOMP,LOGRECNO,STATE,COUNTY,COUNTYCC,NAME
WI,050,00,0000020,55,001,H1,Adams County
WI,050,00,0002473,55,003,H1,Ashland County
WI,050,00,0004915,55,005,H1,Barron County
WI,050,00,0008619,55,007,H1,Bayfield County
WI,050,00,0011746,55,009,H1,Brown County
WI,050,00,0018034,55,011,H1,Buffalo County


In [49]:
#sfa$X <- seq_len(nrow(sfa))
#sfb$Y <- rev(seq_len(nrow(sfa)))
#dt3 <- sfa[sfb]
dt3 <- merge(geo_table, sfa, all.x = TRUE)
dt4 <- merge(dt3, sfb, all.x=TRUE)

#convert back to dataframe to subset using grep. 
df1 <- data.frame(dt4)
df2 <- df1[,c(1:8, grep("^P012[A-I].*", colnames(df1)))]
head(df2)


LOGRECNO,STUSAB.x,SUMLEV,GEOCOMP,STATE,COUNTY,COUNTYCC,NAME,P012A001,P012A002,⋯,P012I040,P012I041,P012I042,P012I043,P012I044,P012I045,P012I046,P012I047,P012I048,P012I049
0000020,WI,050,00,55,001,H1,Adams County,19409,10080,,779,818,327,528,332,508,603,415,309,253
0002473,WI,050,00,55,003,H1,Ashland County,13662,6868,,533,506,188,245,157,182,243,228,242,258
0004915,WI,050,00,55,005,H1,Barron County,44076,21884,,1740,1630,626,849,543,723,970,863,666,870
0008619,WI,050,00,55,007,H1,Bayfield County,13024,6721,,612,670,255,340,213,298,360,252,155,213
0011746,WI,050,00,55,009,H1,Brown County,214415,105076,,8471,7529,2542,3393,1836,2436,3302,2925,2452,2951
0018034,WI,050,00,55,011,H1,Buffalo County,13253,6667,,571,464,199,223,157,172,310,234,206,217


In [167]:
#Labelling functions 
library(gsubfn)
#strapplyc(string, ":(.*)", simplify = TRUE)
desc <- descriptions 

getRace <- function(col){
    string <- col #grep("P012[A-I]", col, value = T)
    table <- gsub(string, substr(string, 1, 5), string) #table label refers to table column in description file
    table1 <- gsub("0", "", table) #cleaned string to specify table id
    race <- as.character(desc$FIELD.NAME[desc$TABLE == table1][1])
    race2 <- strapplyc(race, "\\((.*)\\)", simplify = TRUE)
    return(race2)
}
    
getAge <- function(col){
    string <- desc$FIELD.NAME[desc$FIELD.CODE == col]
    if(grepl("years", string)){
        age1 <- sub(" (years)", "", string)
        age2 <- sub(" to ", "-", age1)
        age <- trimws(age2, which = c("both","left","right"))
        }
    if(grepl("^ *Male:", string)) age <- "ALL"
    if(grepl("^ *Female:", string)) age <- "ALL" 
    if(grepl("^Total:", string)) age <- "ALL"
    return(age)
}

getSex <- function(col){
    num <- as.numeric(strapplyc(col, "^P012[A-I]0([0-4][0-9])", simplify = TRUE))
    if(num > 1 & num < 26) sex <- "MALE"
    if(num >= 26 & num <= 49) sex <- "FEMALE"
    if(num == 1) sex <- "TOTAL"
    return(sex) 
}


In [169]:
#Importing to mongodb:
library(rmongodb)
db <- mongo.create(host = "localhost")
mongo.is.connected(db)
ns <- "test.CountyDemographics3"

importToMongo <- function(doc){
    mongo.insert(db, ns, doc)
}

[1] TRUE

In [170]:
#now we need to classify the variable names (ex. P012002) by using the "descriptions" file before creating a JSON object. 
cnty <- df2

for(r in 1:nrow(cnty)) {
    LOGRECNO <- cnty$LOGRECNO[r]
    STUSAB <- cnty$STUSAB[r]
    SUMLEV <- cnty$SUMLEV[r]
    STATE <- cnty$STATE[r]
    COUNTY <- cnty$COUNTY[r]
    NAME <- cnty$NAME[r]
    FIPS <- paste0(STATE, COUNTY)
    
    for(cc in 9:ncol(cnty)) {
        AGE <- getAge(colnames(cnty[cc]))
        SEX <- getSex(colnames(cnty[cc]))
        RACE <- getRace(colnames(cnty[cc]))
        count <- as.factor(cnty[r,cc])
        #write_JSON(LOGRECNO, STUSAB, SUMLEV, STATE, COUNTY, FIPS, NAME, AGE, SEX, RACE, count)
        
        json_elements = sprintf('{"LOGRECNO":"%s",\n "NAME":"%s",\n "STUSAB": "%s",\n "SUMLEV": "%s", \n "STATE":"%s",\n "COUNTY":"%s",\n "FIPS":"%s",\n "AGE": "%s" ,\n "SEX":"%s",\n "RACE":"%s",\n "COUNT": %s}'
     , LOGRECNO, NAME, STUSAB, SUMLEV, STATE, COUNTY, FIPS, AGE, SEX, RACE, count)
       
        bson_object <- mongo.bson.from.JSON(json_elements)
        if(r < 2 & cc < 10) print(bson_object)
        importToMongo(bson_object)
    next
    }
next
}

	LOGRECNO : 2 	 0000020
	NAME : 2 	 Adams County
	STUSAB : 2 	 WI
	SUMLEV : 2 	 050
	STATE : 2 	 55
	COUNTY : 2 	 001
	FIPS : 2 	 55001
	AGE : 2 	 ALL
	SEX : 2 	 TOTAL
	RACE : 2 	 WHITE ALONE
	COUNT : 16 	 19409


In [166]:
mongo.count(db, ns)

[1] 31752

In [158]:
desc$FIELD.CODE[grep("^ *Male:", desc$FIELD.NAME)]

FIELD CODE 
 [1] P0120002   P0140002   P0430002   P012A002   P012B002   P012C002  
 [7] P012D002   P012E002   P012F002   P012G002   P012H002   P012I002  
[13] PCT0120002 PCT0130002 PCT0210002 PCT0220002 PCT012A002 PCT012B002
[19] PCT012C002 PCT012D002 PCT012E002 PCT012F002 PCT012G002 PCT012H002
[25] PCT012I002 PCT012J002 PCT012K002 PCT012L002 PCT012M002 PCT012N002
[31] PCT012O002 PCT013A002 PCT013B002 PCT013C002 PCT013D002 PCT013E002
[37] PCT013F002 PCT013G002 PCT013H002 PCT013I002 PCT022A002 PCT022B002
[43] PCT022C002 PCT022D002 PCT022E002 PCT022F002 PCT022G002 PCT022H002
[49] PCT022I002 PCO0010002 PCO0020002 PCO0030002 PCO0040002 PCO0050002
[55] PCO0060002 PCO0070002 PCO0080002 PCO0090002 PCO0100002
8913 Levels:  H00010001 H0020001 H0020002 H0020003 H0020004 H0020005 ... PCT022I021